# Problem: Formulate a linear program for maximizing imaginary airline "Spirit Airways" revenue on Mondays.

# Mathematical Formulation

Decision variables:

$ Q_{i} $, $ Y_{i} $ where $i = 1,...,8$

Objective function:

Maximize $\sum \limits _{i=1} ^{8} q_{i}Q_{i} + y_{i} Y_{i}  $

where $ q_{i} $ = Q-class fare price for route i, $ y_{i} $ = Y-class fare price for route i

Subject to the following constraints:

$ Q_{i} ≤ demandQ_{i} $ for all i = 1,...8

$ Y_{i} ≤ demandY_{i} $ for all i = 1,...8

$ Q_{i} + Y_{i} ≤ C $ for all i = 1,...8
where $C$ = aircraft capacity

# Solution

In [ ]:
import pandas as pd ## importing the pandas package 
Spirit= pd.read_csv('/Users/hiradnourbakhsh/Desktop/MGSC 695_R/Assignment 1/Spirit Airways.csv')
print(Spirit)

         Route  Q-class fare price  Y-class fare price  \
0  Bos-Chicago                 200                 230   
1       Bos-SF                 320                 420   
2       Bos-LA                 400                 490   
3   NY-Chicago                 250                 290   
4        NY-SF                 410                 550   
5        NY-LA                 450                 550   
6   Chicago-SF                 200                 230   
7   Chicago-LA                 250                 300   

   Expected demand for Q-class  Expected demand for Y-class  
0                           25                           20  
1                           55                           40  
2                           65                           25  
3                           24                           16  
4                           65                           50  
5                           40                           35  
6                           21             

In [ ]:
# For example, to access column the prices for Q-class:
Spirit['Q-class fare price']

0    200
1    320
2    400
3    250
4    410
5    450
6    200
7    250
Name: Q-class fare price, dtype: int64

In [ ]:
# Import packages

import gurobipy as gp
from gurobipy import *

In [ ]:
# Define model

model = gp.Model('Airlines')

In [ ]:
# set parameters

l = len(Spirit.Route)
I = range(l)

q = list(Spirit['Q-class fare price'].values)
y = list(Spirit['Y-class fare price'].values)

demand_Q = list(Spirit['Expected demand for Q-class'].values)
demand_Y = list(Spirit['Expected demand for Y-class'].values)

# flight routes
Boston = [1,1,1,0,0,0,0,0]
SF = [0,1,0,0,1,0,1,0]
NY = [0,0,0,1,1,1,0,0]
LA = [0,0,1,0,0,1,0,1]
Chicago = [1,0,0,1,0,0,1,1]

In [ ]:
# Define variables

# Decision variables - continuous
Q = model.addVars(l, lb = 0, vtype = GRB.CONTINUOUS, name = ['Q_' + str(i + 1) for i in I])
Y = model.addVars(l, lb = 0, vtype = GRB.CONTINUOUS, name = ['Y_' + str(i + 1) for i in I])

In [ ]:
# setObjective

exp = sum(q[i]*Q[i] + y[i]*Y[i] for i in I)
model.setObjective(exp, GRB.MAXIMIZE)

In [ ]:
# Define constraints
    
# Cannot exceed demand    
for i in I:
    model.addConstr(Q[i] <= demand_Q[i], name = 'Demand_Q_' + str(i + 1))
    model.addConstr(Y[i] <= demand_Y[i], name = 'Demand_Y_' + str(i + 1))
    
# Capacity of aircraft

for i in I:
    model.addConstr(sum((Q[i] + Y[i]) * Boston[i] for i in I) <= 140)
    model.addConstr(sum((Q[i] + Y[i]) * SF[i] for i in I) <= 140)
    model.addConstr(sum((Q[i] + Y[i]) * NY[i] for i in I) <= 140)
    model.addConstr(sum((Q[i] + Y[i]) * LA[i] for i in I) <= 140)
    model.addConstr(sum((Q[i] + Y[i]) * Chicago[i] for i in I) <= 140)

In [ ]:
# optimize

model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 56 rows, 16 columns and 272 nonzeros
Model fingerprint: 0x4b443b9b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 51 rows and 0 columns
Presolve time: 0.02s
Presolved: 5 rows, 16 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8760000e+05   5.887500e+01   0.000000e+00      0s
       8    1.3428000e+05   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.06 seconds (0.00 work units)
Optimal objective  1.342800000e+05


In [ ]:
Q

{0: <gurobi.Var Q_1 (value 14.0)>,
 1: <gurobi.Var Q_2 (value 0.0)>,
 2: <gurobi.Var Q_3 (value 41.0)>,
 3: <gurobi.Var Q_4 (value 20.0)>,
 4: <gurobi.Var Q_5 (value 19.0)>,
 5: <gurobi.Var Q_6 (value 0.0)>,
 6: <gurobi.Var Q_7 (value 11.0)>,
 7: <gurobi.Var Q_8 (value 25.0)>}

In [ ]:
Y

{0: <gurobi.Var Y_1 (value 20.0)>,
 1: <gurobi.Var Y_2 (value 40.0)>,
 2: <gurobi.Var Y_3 (value 25.0)>,
 3: <gurobi.Var Y_4 (value 16.0)>,
 4: <gurobi.Var Y_5 (value 50.0)>,
 5: <gurobi.Var Y_6 (value 35.0)>,
 6: <gurobi.Var Y_7 (value 20.0)>,
 7: <gurobi.Var Y_8 (value 14.0)>}

In [ ]:
print('Sensitivity Analysis (SA)\n ObjVal =', model.ObjVal)
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp']) # Pi = shadow price 
# NOTE: printAttr prints only rows with at least one NON-ZERO value, e.g. model.printAttr('X') prints only non-zero variable values

# pi = Shadow price

Sensitivity Analysis (SA)
 ObjVal = 134280.0

    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
         Q_1           14          200          190          200 
         Q_2            0          320         -inf          360 
         Q_3           41          400          400          410 
         Q_4           20          250          250          260 
         Q_5           19          410          400          450 
         Q_6            0          450         -inf          450 
         Q_7           11          200          160          210 
         Q_8           25          250          240          inf 
         Y_1           20          230          200          inf 
         Y_2           40          420          360          inf 
         Y_3           25          490          400          inf 
         Y_4           16          290          250          inf 
         Y_5           50      

# Optimal Revenue for spirit airways = $134,280.

In [ ]:
model.objVal

134280.0

In [ ]:
print('Optimal allocation decisions for each flight and class:')
for v in model.getVars():
    print(v.varName, '=', v.x)

Optimal allocation decisions for each flight and class:
Q_1 = 14.0
Q_2 = 0.0
Q_3 = 41.0
Q_4 = 20.0
Q_5 = 19.0
Q_6 = 0.0
Q_7 = 11.0
Q_8 = 25.0
Y_1 = 20.0
Y_2 = 40.0
Y_3 = 25.0
Y_4 = 16.0
Y_5 = 50.0
Y_6 = 35.0
Y_7 = 20.0
Y_8 = 14.0


In [ ]:
model.printAttr('x')


    Variable            x 
-------------------------
         Q_1           14 
         Q_3           41 
         Q_4           20 
         Q_5           19 
         Q_7           11 
         Q_8           25 
         Y_1           20 
         Y_2           40 
         Y_3           25 
         Y_4           16 
         Y_5           50 
         Y_6           35 
         Y_7           20 
         Y_8           14 
